In [1]:
from geodesk import *
import geopandas
import pandas as pd
import geojson
import json
import reverse_geocoder as rg
from datetime import date

In [3]:
#####################################################
# Variables to amend before running the code 
#####################################################

# set the filter area to retrieve data features from OpenStreetMap 
filterarea = Features('data/london')

# define project area bounds
west=-0.128 
south=51.587
east=-0.116 
north=51.591

# set the reference details for the project url
url = "https://urlforyourproject.org"

In [4]:
# creates bounding box and coordinate objects
# get all features in the area, show number found
area_bounds = Box(
    west=west, south=south,
    east=east, north=north)
coords = (south,west),(north,east)
thisyear = date.today().year
source = "https://www.openstreetmap.org/"
gazurl = "https://www.visionofbritain.org.uk/data/"

features_in_area = filterarea(area_bounds)
features_in_area.count

2588

In [5]:
# get area data from coordinates
codes = rg.search(coords)
cc = codes[0]['cc']
area = codes[0]['name']
widerarea = codes[0]['admin2']
country = codes[0]['admin1']
print(cc, codes)

Loading formatted geocoded file...
GB [{'lat': '51.57971', 'lon': '-0.12373', 'name': 'Crouch End', 'admin1': 'England', 'admin2': 'Greater London', 'cc': 'GB'}, {'lat': '51.6', 'lon': '-0.11667', 'name': 'Wood Green', 'admin1': 'England', 'admin2': 'Greater London', 'cc': 'GB'}]


In [6]:
# get just the roads in the area
arearoads = features_in_area("w[highway][name]")
arearoads.count

58

In [ ]:
# convert to geojson, then to geopandas, then merge sections of roads together
arearoads.geojson.save('arearoadsgj')
arearoadsgdf = geopandas.read_file('arearoadsgj.geojson')
arearoadsdf = arearoadsgdf.dissolve(by = 'name')

In [8]:
# duplicate index (roadname) column as title, then select needed fields to create df, amend road types to include 'road'
arearoadsdf['title'] = arearoadsdf.index
arearoadsdf['ccodes'] = cc
arearoadsdf['fclasses'] = "R"
arearoadsdf.loc[arearoadsdf['highway'] == 'residential', 'highway'] = 'residential road'
arearoadsdf.loc[arearoadsdf['highway'] == 'primary', 'highway'] = 'primary road'
arearoadsdf.loc[arearoadsdf['highway'] == 'tertiary', 'highway'] = 'tertiary road'
arearoadsdf.head(2)

,geometry,id,highway,source,surface,lit,maxspeed,ref,oneway,foot,...,parking:left:orientation,parking:right:orientation,bicycle_road,flood_prone,parking:both:zone,width,source:width,title,ccodes,fclasses
name,,,,,,,,,,,,,,,,,,,,,
Baden Road,"LINESTRING (-0.1257 51.58847, -0.1255 51.58908)",W8037977,residential road,None,asphalt,yes,20 mph,None,None,None,...,None,None,None,None,None,None,None,Baden Road,GB,R
Beechwood Road,"LINESTRING (-0.12662 51.58922, -0.12566 51.589...",W8037990,residential road,None,None,None,20 mph,None,None,None,...,None,None,None,None,None,None,None,Beechwood Road,GB,R


In [9]:
# add in a description of the roads
arearoadsdf['description'] = "A "+arearoadsdf['highway']+" in "+area+", "+widerarea+", "+country
arearoadslpdf = arearoadsdf[['id','title','description','geometry','ccodes','fclasses']]
arearoadslpdf.head(2)

,id,title,description,geometry,ccodes,fclasses
name,,,,,,
Baden Road,W8037977,Baden Road,"A residential road in Crouch End, Greater Lond...","LINESTRING (-0.1257 51.58847, -0.1255 51.58908)",GB,R
Beechwood Road,W8037990,Beechwood Road,"A residential road in Crouch End, Greater Lond...","LINESTRING (-0.12662 51.58922, -0.12566 51.589...",GB,R


In [10]:
# convert to json, add context
jroads = arearoadslpdf.to_json()
roadsj = json.loads(jroads)
roadsj["@context"] = url

In [11]:
# create a geodataframe of places from the 1900 gazetteer, 
# filter by bounding area, 
# get names and convert to proper nouns
# return as list

streets1900 = list()

gaz = pd.read_csv('data/gb1900_abridged.csv', encoding='utf-16')
gdf = geopandas.GeoDataFrame(gaz, geometry=geopandas.points_from_xy(gaz.longitude, gaz.latitude), crs="EPSG:4326")
area1900 = gdf.cx[east:west,south:north]
names1900 = area1900['final_text'].tolist()
for i in names1900:
    s = i.title().replace("'S","'s")
    streets1900.append(s)

C:\Users\joken\AppData\Local\Temp\ipykernel_3884\623219567.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  gaz = pd.read_csv('data/gb1900_abridged.csv', encoding='utf-16')


In [12]:
# add approximated data for start date of street
# add citations for OpenStreetMap and the GB1900 Gazetteer 
thisyear = date.today().year
for i in roadsj['features']:
    prop = i["properties"]
    rn = prop['title']
    i["@id"] = url+"places/"+prop["id"]
    if rn in streets1900:
        i['when'] = {"timespans": [{ "start": { 'latest':'1900'}}]}
    else:
        i['when'] = {"timespans": [{ "start": { "earliest": "1901" }}]}
    i['names'] = [{"toponym":rn, "lang":"en", "citations": [{"label": "GB1900 Gazetteer (Abridged)", "year": 2018, "@id":gazurl},{"label": "OpenStreetMap", "year": thisyear, "@id":source}] }]
    i['descriptions'] = [{"value": prop['description'], "lang": "en", "source": source }]
    i.pop('id')
    prop.pop('id')
    prop.pop('description')


In [13]:
# write the data our in linked places format
with open('roads.json', 'w', encoding='utf-8') as f:
    json.dump(roadsj, f, ensure_ascii=False, indent=4)


In [ ]:
# print out the json file for checking
roadsj

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'title': 'Baden Road', 'ccodes': 'GB', 'fclasses': 'R'},
   'geometry': {'type': 'LineString',
    'coordinates': [[-0.1257004, 51.5884661], [-0.125505, 51.5890785]]},
   '@id': 'https://campsbourne.uk/memorymapper/places/W8037977',
   'when': {'timespans': [{'start': {'earliest': '1901'}}]},
   'names': [{'toponym': 'Baden Road',
     'lang': 'en',
     'citations': [{'label': 'OpenStreetMap',
       'year': 2025,
       '@id': 'https://www.openstreetmap.org/'}]}],
   'descriptions': [{'value': 'A residential road in Crouch End, Greater London, England',
     'lang': 'en',
     'source': 'https://www.openstreetmap.org/'}]},
  {'type': 'Feature',
   'properties': {'title': 'Beechwood Road', 'ccodes': 'GB', 'fclasses': 'R'},
   'geometry': {'type': 'LineString',
    'coordinates': [[-0.126618, 51.5892173],
     [-0.1256649, 51.5893584],
     [-0.1254461, 51.5893656],
     [-0.1250202, 51.5893694],
     [-0